In [0]:
print("hello varshini")

hello varshini


In [0]:
#Create a Spark Dataframe 
data = [("John", 25), ("Jane", 30), ("Sam", 22)] 
df = spark.createDataFrame(data, ["Name", "Age"])
#Display the Dataframe
display(df)

Name,Age
John,25
Jane,30
Sam,22


In [0]:
#display the data as a bar chart
df.groupBy("Age").count().display()

Age,count
25,1
30,1
22,1


In [0]:
#example code for kafka

#Read streaming data from a socket (simulated source)
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

#Split the lines into words
words = lines.selectExpr("explode(split(value,' ')) as word")

#Count the number of words
wordCounts = words.groupBy("word").count()

# Start the streaming query to console
query = wordCounts.writeStream.outputlode("complete").format("console").start()

#Avait termination (keep it running)

query.waitTermination()

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-1943595768095691>, line 7
      4 lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()
      6 #Split the lines into words
----> 7 words = lines.selectExpr("explode(split(value,"")) as word")
      9 #Count the number of words
     10 wordCounts = words.groupBy("word").count()

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3872, in DataFrame.selectExpr(self, *expr)
   3870 if len(expr) == 1 and isinstanc

In [0]:
#Task 1: Load the Dataset
#Display the first 10 rows and inspect the schema of the dataset.

# File location and type
file_location = "/FileStore/tables/data.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

# Show the first 10 rows
print("Top 10 rows")
df.show(10)

# Inspect the schema
df.printSchema()

#Task 2: Data Cleaning
# Drop rows with null values
df_cleaned = df.dropna()
print("Cleaned data")
df_cleaned.show()

# Filter rows where 'ARR DELAY' is less than or equal to zero
df_filtered = df_cleaned.filter(df_cleaned['_c5'] <= 0)
print("Filtered data")
df_filtered.show()

#Task 3: Aggregation and Summary Statistics
# Group by 'CARRIER' and calculate the average of 'ARR DELAY'
df_grouped = df_filtered.groupBy('_c1').agg({'_c5': 'avg'})
print("Grouped data")
df_grouped.show()

# Calculate min, max, and mean for 'ARR DELAY'
print("Min,Max,Mean")
df_filtered.selectExpr(
    'min(`_c5`) as Min_Delay',
    'max(`_c5`) as Max_Delay',
    'mean(`_c5`) as Mean_Delay'
).show()

# Count the total number of flights per 'CARRIER'
df_grouped_count = df_filtered.groupBy('_c1').count()
print("Flight count")
df_grouped_count.show()

# Histogram for the distribution of 'ARR DELAY'
print("Histogram")
df_filtered.select('_c5').show()



_c0,_c1,_c2,_c3,_c4,_c5
FL_DATE,CARRIER,ORIGIN,DEST,DEP_DELAY,ARR_DELAY
2023-09-01,AA,ATL,DFW,5,10
2023-09-01,UA,LAX,JFK,-3,0
2023-09-01,DL,SFO,ORD,7,15
2023-09-02,AA,DFW,LAX,0,-5
2023-09-02,UA,JFK,ATL,-2,0
2023-09-02,DL,ORD,LAX,20,30
2023-09-03,AA,LAX,SFO,10,12
2023-09-03,UA,ATL,ORD,0,-10
2023-09-03,DL,SFO,JFK,5,25


Top 10 rows
+----------+-------+------+----+---------+---------+
|       _c0|    _c1|   _c2| _c3|      _c4|      _c5|
+----------+-------+------+----+---------+---------+
|   FL_DATE|CARRIER|ORIGIN|DEST|DEP_DELAY|ARR_DELAY|
|2023-09-01|     AA|   ATL| DFW|        5|       10|
|2023-09-01|     UA|   LAX| JFK|       -3|        0|
|2023-09-01|     DL|   SFO| ORD|        7|       15|
|2023-09-02|     AA|   DFW| LAX|        0|       -5|
|2023-09-02|     UA|   JFK| ATL|       -2|        0|
|2023-09-02|     DL|   ORD| LAX|       20|       30|
|2023-09-03|     AA|   LAX| SFO|       10|       12|
|2023-09-03|     UA|   ATL| ORD|        0|      -10|
|2023-09-03|     DL|   SFO| JFK|        5|       25|
+----------+-------+------+----+---------+---------+
only showing top 10 rows

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)

Clea

In [0]:


### **Task 3: Data Cleaning**

#1. **Handle missing values**: Drop rows with missing values from the dataset.
  # - Use `.na.drop()` to remove rows containing `null` values.
  # - Verify if there are any null values left using `.filter()`.

   #Example:
   #```python
df_cleaned = df.na.drop()
df_cleaned.show()
   #```

#2. **Filter rows**: Create a filtered DataFrame where arrival delays are greater than `0`.
   
 #  Example:
  # ```python
df_filtered = df.filter(df['_c5'] > 0)
df_filtered.show()
   #```

#---

### **Task 4: Aggregating and Summarizing the Data**

#1. **Find the average arrival delay by airline**:
 #  - Group by `CARRIER` and calculate the average of `ARR_DELAY`.

  # Example:
   #```python
df.groupBy("_c1").agg({"_c5": "avg"}).show()
#   ```

#2. **Count the number of flights per airline**:
 #  - Group by `CARRIER` and count the total number of flights.

  # Example:
   #```python
df.groupBy("_c5").count().show()
   #```

#3. **Find the minimum and maximum delay** for all flights:
 #  - Use `.agg()` to calculate both the minimum and maximum delay.

  # Example:
  # ```python
df.agg({"_c5": "min", "_c5": "max"}).show()
   #```

#---

### **Task 5: Visualizing the Data**

#1. **Plot the average delay per airline** using Databricks’ built-in visualization tools:
   #- Use `display()` to visualize the result from the average delay aggregation.

   ##Example:
   #```python
display(df.groupBy("_c1").agg({"_c5": "avg"}))
   #```

#2. **Visualize flight count by airline** using a bar chart:
 #  - Use the `display()` function and convert the table into a bar chart in the UI.

   #Example:
  # ```python
display(df.groupBy("_c1").count())
   #```

#3. **Plot the distribution of arrival delays** using a histogram:
   #- Group by `ARR_DELAY` and count, then use the Databricks visualization tool to create a histogram.

   #Example:
   #```python
display(df.groupBy("_c5").count())
   #```

#---


+----------+-------+------+----+---------+---------+
|       _c0|    _c1|   _c2| _c3|      _c4|      _c5|
+----------+-------+------+----+---------+---------+
|   FL_DATE|CARRIER|ORIGIN|DEST|DEP_DELAY|ARR_DELAY|
|2023-09-01|     AA|   ATL| DFW|        5|       10|
|2023-09-01|     UA|   LAX| JFK|       -3|        0|
|2023-09-01|     DL|   SFO| ORD|        7|       15|
|2023-09-02|     AA|   DFW| LAX|        0|       -5|
|2023-09-02|     UA|   JFK| ATL|       -2|        0|
|2023-09-02|     DL|   ORD| LAX|       20|       30|
|2023-09-03|     AA|   LAX| SFO|       10|       12|
|2023-09-03|     UA|   ATL| ORD|        0|      -10|
|2023-09-03|     DL|   SFO| JFK|        5|       25|
|2023-09-04|     AA|   JFK| LAX|        0|        0|
|2023-09-04|     UA|   ORD| ATL|       15|       20|
|2023-09-04|     DL|   LAX| SFO|       -5|      -10|
|2023-09-05|     AA|   LAX| JFK|       20|       25|
|2023-09-05|     UA|   DFW| ATL|        0|        0|
|2023-09-05|     DL|   JFK| LAX|       10|    

_c1,avg(_c5)
UA,2.0
AA,8.4
DL,15.0
CARRIER,null


_c1,count
UA,5
AA,5
DL,5
CARRIER,1


_c5,count
15,2
30,1
0,4
-10,2
25,2
ARR_DELAY,1
20,1
10,1
12,1
-5,1


In [0]:
print("hii")

hii
